In [115]:
import pandas as pd
from sodapy import Socrata
from datetime import datetime
import pickle
from geopy import distance
import numpy as np
import time

In [116]:
start = time.time()

dict_dfs={"grad":["nb39-jx2v"], "trees":["uvpi-gqnh"],"crashes":["h9gi-nx95"],"pc":["cuzb-dmcd"],
          "shootings":["833y-fsy8"], "arrests":["8h9b-rp9u"],"programs":["mbd7-jfnc"],"bins":["sxx4-xhzg"]}
for key in dict_dfs:
    x=dict_dfs[key][0]
    dict_dfs[key]={"key":x,"last_updated":0,"data":""}
    
dict_dfs

{'grad': {'key': 'nb39-jx2v', 'last_updated': 0, 'data': ''},
 'trees': {'key': 'uvpi-gqnh', 'last_updated': 0, 'data': ''},
 'crashes': {'key': 'h9gi-nx95', 'last_updated': 0, 'data': ''},
 'pc': {'key': 'cuzb-dmcd', 'last_updated': 0, 'data': ''},
 'shootings': {'key': '833y-fsy8', 'last_updated': 0, 'data': ''},
 'arrests': {'key': '8h9b-rp9u', 'last_updated': 0, 'data': ''},
 'programs': {'key': 'mbd7-jfnc', 'last_updated': 0, 'data': ''},
 'bins': {'key': 'sxx4-xhzg', 'last_updated': 0, 'data': ''}}

In [117]:
def get_data(data_dict):
    data_url = "data.cityofnewyork.us"
    token = "HnDwBjZUCabVQojpbl6aIDrDI"
    client = Socrata(data_url,token)
    for key in data_dict:
        data_set=data_dict[key]["key"]
        meta = client.get_metadata(data_set)
        if meta["viewLastModified"]>data_dict[key]["last_updated"]:
            data_dict[key]["last_updated"]=meta["viewLastModified"]
            data_dict[key]["data"]=client.get(data_set, limit=10000)
            lastUpdated=True
        print(key + " done")      
    
     

get_data(dict_dfs)


grad done
trees done
crashes done
pc done
shootings done
arrests done
programs done
bins done


In [118]:
def get_dataframes(data_dict):
    for key in data_dict:
        if key=="addresses":
            continue
        df=pd.DataFrame(data_dict[key]["data"])
        if "latitude" in df.columns:
            df["location"]=list(zip(df["latitude"].to_list(),df["longitude"].to_list()))
        if "lat" in df.columns:
            df["location"]=list(zip(df["lat"].to_list(),df["lon"].to_list()))
        data_dict[key]["df"]=df
        print(key + " done")
    df=pd.read_csv("data/2016_DOE_High_School_Directory-2.csv")
    df["location"]=list(zip(df["Latitude"].to_list(),df["Longitude"].to_list()))
    df=df[["dbn","school_name","borough","location"]]
    data_dict["grad"]["df"]=data_dict["grad"]["df"].merge(df,on="dbn",how="left")
    data_dict["scatter"]={"df":df}
get_dataframes(dict_dfs)

grad done
trees done
crashes done
pc done
shootings done
arrests done
programs done
bins done


In [119]:
df_school=dict_dfs["grad"]["df"]

df_school[df_school.location.notnull()]

,dbn,school_name_x,demographic_category,demographic_variable,cohort_year,cohort,total_cohort,total_grads,total_grads_of_cohort,total_regents,...,local,local_of_cohort,local_of_grads,still_enrolled,still_enrolled_of_cohort,dropped_out,dropped_out_of_cohort,school_name_y,borough,location
0,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2013,4 year August,36,25,69.4,23,...,2,5.6,8.0,3,8.3,7,19.4,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)"
1,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2012,4 year August,44,24,54.5,20,...,4,9.1,16.7,10,22.7,10,22.7,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)"
2,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2011,4 year August,73,46,63.0,41,...,5,6.8,10.9,18,24.7,7,9.6,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)"
3,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2010,4 year August,61,26,42.6,26,...,0,0.0,0.0,18,29.5,17,27.9,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)"
4,01M292,ORCHARD COLLEGIATE ACADEMY,All Students,All Students,2009,4 year August,85,49,57.6,44,...,5,5.9,10.2,28,32.9,8,9.4,Henry Street School for International Studies,Manhattan,"(40.713684, -73.986336)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2006,5 year June,92,80,87.0,60,...,20,21.7,25.0,5,5.4,6,6.5,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)"
9996,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2005,5 year June,94,79,84.0,55,...,24,25.5,30.4,10,10.6,4,4.3,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)"
9997,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2004,5 year June,84,80,95.2,48,...,32,38.1,40.0,1,1.2,3,3.6,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)"
9998,11X275,HIGH SCHOOL OF COMPUTERS AND TECHNOLOGY,All Students,All Students,2003,5 year June,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,High School of Computers and Technology,Bronx,"(40.875953, -73.86197)"


In [120]:
def get_distance(loc1,loc2):
    try:
        return distance.distance(loc1,loc2).meters
    except:
        return np.nan
    

def get_numFeat(dict_dfs):
    dbns=dict_dfs["grad"]["df"]["dbn"].to_list()
    df_school=dict_dfs["grad"]["df"]
    df_school=df_school[df_school.location.notnull()]
    dbns=set(df_school["dbn"].to_list())
    dict_dbn={}
    dict_keys={}
    dict_feat={}
    dict_dfs["scatter"]["feat"]=""
    
    for key in dict_dfs:
        if key != "scatter":
            dict_dbn={}
            print( "\n\n\n",key)
            if key != "grad":
                df=dict_dfs[key]["df"]
                df=df[df.location.notnull()]
                for dbn in dbns:
                    locSchool=df_school.loc[df_school["dbn"]==dbn,"location"].to_list()[0]
                    df["distance"]=df["location"].apply(lambda x: get_distance(locSchool,x))
                    numFeat=(df["distance"].values<500).sum()
                    dict_dbn[dbn]=numFeat
                dict_keys[key]=dict_dbn
    #dict_feat["feat"]=dict_keys
    dict_dfs["scatter"]["feat"]=dict_keys
                   
                    
            
 
    



get_numFeat(dict_dfs)

end=time.time()

print("/n/n",end-start)




 grad



 trees



 crashes



 pc



 shootings



 arrests



 programs



 bins
/n/n 753.4946410655975


In [125]:
def get_scatterDf(data_dict):
    dfScatter=data_dict["scatter"]["df"]
    dfFeat=pd.DataFrame.from_dict(data_dict["scatter"]["feat"])
    dfFeat.reset_index(inplace=True)
    dfFeat=dfFeat.rename(columns={"index":"dbn"})
    dfScatter=dfScatter.merge(dfFeat,on="dbn",how="left")
    return dfScatter



x=get_scatterDf(dict_dfs)

x.to_csv("data/dataForScatter.csv")

In [122]:
dict_dfs["scatter"]["feat"]

{'trees': {'10X237': 1,
  '02M534': 19,
  '09X365': 40,
  '04M372': 3,
  '06M468': 1,
  '02M600': 10,
  '09X329': 2,
  '02M655': 15,
  '04M680': 19,
  '02M475': 0,
  '02M294': 26,
  '08X348': 9,
  '05M157': 0,
  '02M376': 8,
  '05M367': 3,
  '03M494': 88,
  '04M495': 30,
  '08X312': 0,
  '10X433': 20,
  '07X547': 0,
  '09X241': 12,
  '02M546': 9,
  '09X413': 2,
  '02M374': 9,
  '10X368': 2,
  '06M540': 2,
  '08X349': 9,
  '02M412': 5,
  '02M425': 10,
  '02M630': 12,
  '02M507': 78,
  '02M542': 57,
  '02M393': 78,
  '10X225': 120,
  '06M552': 0,
  '06M462': 1,
  '10X213': 1,
  '10X353': 5,
  '10X141': 3,
  '02M316': 7,
  '10X434': 97,
  '10X440': 5,
  '05M670': 1,
  '02M399': 9,
  '03M415': 57,
  '09X543': 8,
  '05M369': 3,
  '08X269': 0,
  '02M615': 8,
  '02M308': 26,
  '11X513': 3,
  '07X259': 0,
  '02M414': 5,
  '09X564': 2,
  '07X500': 0,
  '08X559': 4,
  '10X442': 20,
  '02M529': 15,
  '07X427': 0,
  '09X412': 2,
  '04M435': 0,
  '04M409': 0,
  '03M492': 88,
  '05M304': 2,
  '10X44

In [123]:
dict

dict